## **1. Setup**

The project requires several packages that need to be installed into Workspace:

- Langchain: is a framework for developing generative Al applications.
- yt_dip: lets you download YouTube vide
- tiktoken: converts text into tokens.
- docarray: makes it easier to work with multi-model data (in this case mixing audio and text).

In [1]:
# !pip install -r requirements.txt

In [2]:
#!pip install openai

In [3]:
#!pip install python-dotenv

In [4]:
#!pip install -U langchain-community

In [5]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [6]:
#!pip install transformers soundfile langchain docarray

In [7]:
#!pip install librosa


In [8]:
import yt_dlp as youtube_dl
from yt_dlp import DownloadError
import os
import glob
import torch
import soundfile as sf
import librosa
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.chat_history import InMemoryChatMessageHistory
from dotenv import load_dotenv


### **YouTube Audio Extraction & MP3 Conversion**
This snippet specifies an output folder and a target YouTube URL, then configures yt_dlp to:

1. Download the highest-quality audio stream available.

2. Use FFmpeg to extract and convert that stream into a 192 kbps MP3.

3. Name the resulting file after the video’s title.

4. Enable verbose logging so you can see detailed progress.

It wraps the download call in a try/except block to catch and report any DownloadError that might occur.

In [9]:
output_dir = "audio_files/"
youtube_url ="https://youtu.be/4h9lQfYLOZU?si=4Z4RCfJjaAdAp2e1"

In [10]:
# Configuration for yt_dlp
def download_audio(url: str) -> str:
    
    ydl_config = {
        "format": "bestaudio/best",
        # List of post-processing steps; each dict represents one processor
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",   # Extract audio from the downloaded file
                "preferredcodec": "mp3",       # Convert audio to MP3 format
                "preferredquality": "192"      # Set audio quality to 192 kbps
            }
        ],
        # Template for naming the output file: "<video title>.<extension>"
        "outtmpl": "output/%(title)s.%(ext)s",
        "verbose": True  # Enable detailed logging
    }

    try:
        # Initialize the downloader with the specified config and start download
        with youtube_dl.YoutubeDL(ydl_config) as ydl:
            info = ydl.extract_info(url, download=True)
    except DownloadError as e:
        # Print any download errors that occur
        print("DownloadError:", e)
    
    return f"Downloaded: output_audio/{info['id']}.mp3"

### **Batch Transcription of MP3 Files with Whisper**

1. Detects whether to run on GPU (CUDA) or CPU and sets the optimal PyTorch data type for memory efficiency.

2. Loads the openai/whisper-large-v3 model (using safetensors for faster, lower-memory loading) and moves it to the chosen device.

3. Loads the accompanying processor (tokenizer + feature extractor).

4. Builds an ASR pipeline around that model + processor.

5. Uses glob to collect all .mp3 files in the specified output_dir and validates that at least one file exists.

6. Defines a single transcript output path (files/transcripts/transcript.txt) and ensures its folder is created.

7. Iterates over each MP3, reads the audio into an array, runs the ASR pipeline to generate text, and writes the resulting transcript (overwriting on each loop) to the designated text file.

In [11]:
# # 1. Figure out if I can use GPU
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# #    and choose torch dtype to save memory on GPU
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# # 2. Pick the Whisper model I want
# model_id = "openai/whisper-small-v3"

# # 3. Load the model weights (using safetensors to speed things up)
# model = AutoModelForSpeechSeq2Seq.from_pretrained(
#     model_id,
#     torch_dtype=torch_dtype,
#     low_cpu_mem_usage=True,
#     use_safetensors=True
# )
# # move the model to the right device
# model.to(device)


In [12]:
# # 4. Load the processor that has both tokenizer & feature extractor
# processor = AutoProcessor.from_pretrained(model_id)


In [13]:
# # 5. Build the ASR pipeline with my model and processor
# pipe = pipeline(
#     "automatic-speech-recognition",
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     torch_dtype=torch_dtype,
#     device=device,
# )

In [14]:
print("Current working dir:", os.getcwd())
print("ffmpeg exists:", os.path.exists(r"C:\ffmpeg\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe"))
print("ffprobe exists:", os.path.exists(r"C:\ffmpeg\ffmpeg-7.1.1-full_build\bin\ffprobe.exe"))

Current working dir: d:\Ironhack\Final_project\Multimodal-AI-ChatBot-for-YouTube-Video-QA
ffmpeg exists: True
ffprobe exists: True


In [15]:
from pydub import AudioSegment
from openai import OpenAI
import ffmpeg

# Load your OpenAI API Key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# AudioSegment.converter = r"C:\ffmpeg\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe"
# AudioSegment.ffprobe = r"C:\ffmpeg\ffmpeg-7.1.1-full_build\bin\ffprobe.exe"



AudioSegment.ffmpeg = r"C:\ffmpeg\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe"
AudioSegment.converter = r"C:\ffmpeg\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe"




# Paths
input_folder = "audio_files/"
chunk_folder = "chunks"
output_file = "files/transcripts/final_transcript.txt"

os.makedirs(chunk_folder, exist_ok=True)
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Settings
chunk_length = 10 * 60 * 1000  # 10 minutes in milliseconds

# Find MP3 files
audio_files = glob.glob(os.path.join(input_folder, "*.mp3"))
if not audio_files:
    raise ValueError("❌ No MP3 files found in the folder!")

# Process files one by one
for audio_path in audio_files:
    base_name = os.path.splitext(os.path.basename(audio_path))[0]
    print(f"\n🎧 Processing: {base_name}")

    # Load audio file
    audio = AudioSegment.from_file(audio_path)
    total_length = len(audio)
    num_chunks = (total_length // chunk_length) + 1

    for i in range(num_chunks):
        start = i * chunk_length
        end = min(start + chunk_length, total_length)
        chunk = audio[start:end]

        chunk_filename = f"{base_name}_chunk_{i}.mp3"
        chunk_path = os.path.join(chunk_folder, chunk_filename)
        chunk.export(chunk_path, format="mp3")

        print(f"🔹 Transcribing chunk {i+1}/{num_chunks}...")

        # Transcribe
        with open(chunk_path, "rb") as f:
            response = client.audio.transcriptions.create(
                file=f,
                model="whisper-1"
            )

        # Save transcript
        with open(output_file, "a", encoding="utf-8") as out:
            out.write(f"## {chunk_filename}\n")
            out.write(response.text + "\n\n")

    print(f"✅ Done with {base_name}")

print(f"\n📄 Full transcript saved to: {output_file}")



🎧 Processing: IELTS Listening Practice ｜ What You NEED to Know!
🔹 Transcribing chunk 1/2...
🔹 Transcribing chunk 2/2...
✅ Done with IELTS Listening Practice ｜ What You NEED to Know!

🎧 Processing: IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words
🔹 Transcribing chunk 1/3...
🔹 Transcribing chunk 2/3...
🔹 Transcribing chunk 3/3...
✅ Done with IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words

📄 Full transcript saved to: files/transcripts/final_transcript.txt


### **Load Transcript File into LangChain Documents** 

Uses LangChain’s TextLoader to read transcript.txt and convert it into a list of Document objects for downstream processing.

In [16]:
# Create a new instance of the TextLoader class, specifying the directory containing the text files
loader = TextLoader("./files/transcripts/final_transcript.txt")

# Load the documents from the specified directory using the TextLoader instance
docs = loader.load()

In [17]:
docs[0]

Document(metadata={'source': './files/transcripts/final_transcript.txt'}, page_content="## IELTS Listening Practice ï½œ What You NEED to Know!_chunk_0.mp3\nHi everyone, Chris here from IELTSadvantage.com with another lesson, and today what we're gonna focus on is how to practice IELTS listening. So what we're gonna do is look at why doing lots of practice tests is a terrible idea. In fact, this is the worst thing you could do. If you think that just doing lots and lots and lots of practice tests is gonna help you get a higher score, you are wrong. But what then I'm gonna show you is three better ways to practice that will actually improve your scores because at the end of the day, what are we doing? We're helping you improve your scores. So we're only gonna teach you the things that work and make you aware of the things that don't work. And these three ways are totally free and you can do them at home by yourself without a teacher. So they're absolutely brilliant. So number one, why do

In [18]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=100)
docs = text_splitter.split_documents(docs)

Created a chunk of size 9671, which is longer than the specified 600
Created a chunk of size 3629, which is longer than the specified 600
Created a chunk of size 8372, which is longer than the specified 600
Created a chunk of size 8363, which is longer than the specified 600
Created a chunk of size 3155, which is longer than the specified 600
Created a chunk of size 9671, which is longer than the specified 600
Created a chunk of size 3629, which is longer than the specified 600
Created a chunk of size 8379, which is longer than the specified 600
Created a chunk of size 8363, which is longer than the specified 600
Created a chunk of size 3155, which is longer than the specified 600
Created a chunk of size 9711, which is longer than the specified 600
Created a chunk of size 3640, which is longer than the specified 600
Created a chunk of size 8372, which is longer than the specified 600
Created a chunk of size 8363, which is longer than the specified 600
Created a chunk of size 3155, whic

In [19]:
docs

[Document(metadata={'source': './files/transcripts/final_transcript.txt'}, page_content="## IELTS Listening Practice ï½œ What You NEED to Know!_chunk_0.mp3\nHi everyone, Chris here from IELTSadvantage.com with another lesson, and today what we're gonna focus on is how to practice IELTS listening. So what we're gonna do is look at why doing lots of practice tests is a terrible idea. In fact, this is the worst thing you could do. If you think that just doing lots and lots and lots of practice tests is gonna help you get a higher score, you are wrong. But what then I'm gonna show you is three better ways to practice that will actually improve your scores because at the end of the day, what are we doing? We're helping you improve your scores. So we're only gonna teach you the things that work and make you aware of the things that don't work. And these three ways are totally free and you can do them at home by yourself without a teacher. So they're absolutely brilliant. So number one, why d

### **Build & Configure the RetrievalQA Pipeline**



1. Creates a DocArrayInMemorySearch index from your docs, embedding each with OpenAI’s embeddings API.



In [20]:
# Load .env into environment
_ = load_dotenv()


In [21]:
#print("API KEY:", os.getenv("OPENAI_API_KEY"))


In [22]:
#Create a new DoCArrayInMenorySearch Instance from the Specified documents and embeddings
# vectorstore = DocArrayInMemorySearch.from_documents(
# docs,
# OpenAIEmbeddings())

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
from langchain.vectorstores import FAISS

#from langchain.vectorstores import Chroma
# vectorstore= Chroma.from_documents(
#     docs,
#     embeddings,
#     collection_name="ielts-rag",
#     persist_directory="./chroma_db")
vectorstore = FAISS.from_documents(docs, embeddings)

C:\Users\Lsula\AppData\Local\Temp\ipykernel_20596\3734980728.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


2. Converts that index into a retriever for semantic search.

3. Instantiates a ChatOpenAI model with zero temperature for deterministic responses.

In [23]:
#Convert DocArrayInMemorySearch instance to a retriever
# Get threshold-filtered docs first
# Fallback to top-k if no threshold matches
#retriever.get_relevant_documents(question) to see actual scores
retriever = vectorstore.as_retriever(
)


#Create a new chatOpenAi 
llm = ChatOpenAI(temperature = 0.0)

C:\Users\Lsula\AppData\Local\Temp\ipykernel_20596\3236617018.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature = 0.0)


4. Builds a RetrievalQA chain (using the “stuff” strategy) that ties together the LLM and retriever, with verbose=True to print intermediate debug info.

In [29]:
from langchain.prompts import PromptTemplate

# Custom Prompt Template for Strict QA
STRICT_QA_PROMPT = PromptTemplate(
    template="""Answer ONLY from this transcript. If not found, say "This information is not mentioned in the transcript of the video".
    
    Transcript: {context}
    
    Question: {question}
    
    Answer:""",
    input_variables=["context", "question"]
)

In [30]:
# Create a new RetrievalQA instance with the specified parameters
qa_stuff = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever= retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": STRICT_QA_PROMPT},
verbose=True)


In [26]:
#qa_stuff?

### **Multi-Tool Conversational Agent with Memory**

Sets up three custom tools (transcript Q&A, summarization, and YouTube audio download), configures a conversation buffer to remember past messages, and initializes a LangChain agent that can use these tools interactively while preserving chat history.

In [31]:
#define a summarization function and wrap it as a tool
def summarize_transcript(text: str) -> str:
    output = qa_stuff({"query": f"Summarize this:\n\n{text}"})
    answer = output["result"]
    sources = "\n\n".join([doc.page_content for doc in output['source_documents']])
    return f"Answer:\n{answer}\n\nSources:\n{sources}"


# def run_TranscriptQA(symptoms: str) -> str:
#     output = qa_stuff({"query": f"answer this:\n\n{symptoms}"})
#     answer = output["result"]
#     sources = "\n\n".join([doc.page_content for doc in output['source_documents']])
#     return f"Answer:\n{answer}\n\nSources:\n{sources}"

def run_TranscriptQA(question: str) -> str:
    output = qa_stuff({"query": question})
    answer = output["result"]
    sources = "\n\n".join([doc.page_content for doc in output['source_documents']])
    if not sources.strip() or not answer.strip() or "not mentioned" in answer.lower():
        return "Sorry, this information is not in the video transcript."
    return f"Answer:\n{answer}\n\nSources:\n{sources}"


In [33]:
tools = [Tool(
    name="TranscriptQA",
    func=run_TranscriptQA,
    description="The user will ask about the video but you will Answer questions strictly based on the video transcript. If the information is not in the transcript, no answer will be provided.No general knowledge.",
    return_direct=True),
   
         
    Tool(
    name="TranscriptSummarizer",
    func=summarize_transcript,
    description="Generate a concise summary of a given transcript text.",
    return_direct=True
    
)]


In [ ]:
#et up a downloader tool to grab YouTube audio and convert it to MP3
# downloader_tool = Tool(
#     name="YouTubeDownloader",
#     func=download_audio,
#     description="Download and convert a YouTube URL to an MP3 file."
# )


In [34]:
session_store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """Ensures each session has its own message history"""
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

In [35]:
import uuid
session_id = str(uuid.uuid4())  # e.g. "4f9b8a2e-1c3d-4f5a-9e6b-7d8f0a1b2c3d"

#configure a memory buffer to keep the full conversation history
memory = get_session_history(session_id)

In [36]:
chat_history = InMemoryChatMessageHistory()

memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=chat_history,
    return_messages=True
)


C:\Users\Lsula\AppData\Local\Temp\ipykernel_20596\4058408968.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [37]:
# I initialize the agent with my tools, the LLM, and the memory buffer
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="chat-conversational-react-description",
    memory=memory,
    handle_parsing_errors=True,
    verbose=True,early_stopping_method="generate",
    agent_kwargs={
        "prefix": """You are an IELTS and English expert who only answers based on transcripts. 
        Follow these rules:
        1. If question requires general knowledge, say "I only answer using the transcripts"
        2. Never make up answers
        3. Use only the tools provided to you4.
        4. nogeneral knowledge""",
    }
)

C:\Users\Lsula\AppData\Local\Temp\ipykernel_20596\3675263955.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
#agent?

### **Execute RetrievalQA Query & Display Answer**
1. Defines the user’s question as the query string.

2. Runs that query through the previously configured RetrievalQA chain (qa_stuff), which retrieves relevant passages and then generates an answer.

3. Prints out the final response text to the console.

In [42]:
# set the query
query = "What is this the video about?"

#run the query 
response = agent.invoke(query)

#print response
print(response)




> Entering new AgentExecutor chain...
```json
{
    "action": "TranscriptSummarizer",
    "action_input": "IELTS Listening Practice video transcript"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: Answer:
The key to improving IELTS scores is to listen reflectively, use genuine practice tests, evaluate mistakes honestly, identify patterns, and take action to improve in specific areas such as spelling, vocabulary, and question types. By focusing on weaknesses and actively listening to enjoyable content, test-takers can see significant improvements in their scores.

Sources:
## IELTS Listening Practice ï½œ What You NEED to Know!_chunk_1.mp3
important one in terms of improving your IELTS scores is listen reflectively. So this is when you do practice tests. So you get a practice test make sure it's a real genuine practice test. Do not use these fake tests that you'll find online. They're only there so that they can they can get clicks on their website. 90% ove

In [41]:
query = "2+2"

#run the query 
response = agent.invoke(query)

#print response
print(response)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "4"
}
```

> Finished chain.
{'input': '2+2', 'chat_history': [HumanMessage(content='What is this the video about?', additional_kwargs={}, response_metadata={}), AIMessage(content="Answer:\nIELTS Listening Practice\n\nSources:\n## IELTS Listening Practice ï½œ What You NEED to Know!_chunk_1.mp3\nimportant one in terms of improving your IELTS scores is listen reflectively. So this is when you do practice tests. So you get a practice test make sure it's a real genuine practice test. Do not use these fake tests that you'll find online. They're only there so that they can they can get clicks on their website. 90% over 90% of practice tests you'll find online are fake tests. They're not made by Cambridge. The only tests you should use are the Cambridge tests and because they're created by the people who create the test and they know what they're doing. So what you do is you do the test under e

In [40]:
query = "what is the capital of ksa"

#run the query 
response = agent.run(query)

#print response
print(response)



> Entering new AgentExecutor chain...
```json
{
    "action": "TranscriptQA",
    "action_input": "What is the capital of KSA?"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: Sorry, this information is not in the video transcript.


> Finished chain.
Sorry, this information is not in the video transcript.
